In [2]:
# Unicode Handling
from __future__ import unicode_literals
import codecs

import numpy as np
import gensim

# spacy is used for pre-processing and traditional NLP
import spacy
from spacy.en import English

# Gensim is used for LDA and word2vec
from gensim.models.word2vec import Word2Vec

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
# Loading the tweet data
filename = '../../assets/dataset/captured-tweets.txt'
tweets = []
for tweet in codecs.open(filename, 'r', encoding="utf-8"):
    tweets.append(tweet)
# Setting up spacy
nlp_toolkit = English()

## Exercise 1a

Write a function that can take a take a sentence parsed by `spacy` and identify if it mentions a company named 'Google'. Remember, `spacy` can find entities and codes them as `ORG` if they are a company. Look at the slides for class 12 if you need a hint:

### Bonus (1b)

Parameterize the company name so that the function works for any company.

In [4]:
def mentions_company(parsed):
    for entity in parsed.ents:
        if entity.text == "Google" and entity.label_ == 'ORG':
            return True
    return False

# 1b

def mentions_company(parsed, company='Google'):
    for entity in parsed.ents:
        if entity.text == company and entity.label_ == 'ORG':
            return True
    return False

In [5]:
#always test your function!
sentence = 'This company is totally Google and not IBM'
parsed=nlp_toolkit(sentence)

In [6]:
mentions_company(parsed,company='Google')

True

## Exercise 1c

Write a function that can take a sentence parsed by `spacy` 
and return the verbs of the sentence (preferably lemmatized)

In [9]:
def get_actions(parsed):
    actions = []
    for el in parsed:
        if el.pos == spacy.parts_of_speech.VERB:
            actions.append(el.text)
    return actions

In [10]:
# Testing again
sentence = 'He merrily ran down the lane to live another day'
parsed=nlp_toolkit(sentence)

In [11]:
get_actions(parsed)

['ran', 'live']

## Exercise 1d
For each tweet, parse it using spacy and print it out if the tweet has 'taking' or 'joins' as a verb. You'll need to use your `mentions_company` and `get_actions` functions.

In [12]:
for tweet in tweets:
    parsed = nlp_toolkit(tweet)
    if mentions_company(parsed, 'Google'):
        actions = get_actions(parsed)
        if 'takes' in actions or 'joins' in actions:
            print(tweet)

.@sardesairajdeep It takes 58 minutes now as per Google maps...unless of course Google is a Modi agent. https://t.co/flMt7tFYwu

RT @MFordFuture: Google takes quantum leap into artificial intelligence

RT @ramprasad_c: .@sardesairajdeep It takes 58 minutes now as per Google maps...unless of course Google is a Modi agent. https://t.co/flMt7…

RT @ramprasad_c: .@sardesairajdeep It takes 58 minutes now as per Google maps...unless of course Google is a Modi agent. https://t.co/flMt7…

RT @ramprasad_c: .@sardesairajdeep It takes 58 minutes now as per Google maps...unless of course Google is a Modi agent. https://t.co/flMt7…

RT @ramprasad_c: .@sardesairajdeep It takes 58 minutes now as per Google maps...unless of course Google is a Modi agent. https://t.co/flMt7…

RT @ramprasad_c: .@sardesairajdeep It takes 58 minutes now as per Google maps...unless of course Google is a Modi agent. https://t.co/flMt7…

RT @ramprasad_c: .@sardesairajdeep It takes 58 minutes now as per Google maps...unless o

## Exercise 1e
Write a function that identifies countries - HINT: the entity label for countries is GPE (or GeoPolitical Entity)



In [13]:
def mentions_country(parsed, country):
    for entity in parsed.ents:
        if entity.text == country and entity.label_ == 'GPE':
            return True
    return False

In [14]:
#Testing it out
sentence = 'A man, a plan, a canal, Panama'
parsed=nlp_toolkit(sentence)
mentions_country(parsed, country='Panama')


True

## Exercise 1f

Re-run (d) to find country tweets that discuss 'Iran' announcing or releasing.


In [15]:
for tweet in tweets:
    parsed = nlp_toolkit(tweet)

    if mentions_country(parsed, 'Iran'):
        actions = get_actions(parsed)
        if 'release' in actions or 'announce' in actions:
            print(tweet)

RT @cerenomri: "Literally every US ally in Mideast is on brink of hot war w/ Iran, so we're going to release $100 billion to Iran this mont…

GOBE! Iran warns Nigeria to release Shiite leader El-Zakzaky - SEE https://t.co/TRshnC6sVU

GOBE! Iran warns Nigeria to release Shiite leader El-Zakzaky - SEE https://t.co/SlvcQtk3vE

RT @cerenomri: "Literally every US ally in Mideast is on brink of hot war w/ Iran, so we're going to release $100 billion to Iran this mont…

Hhmmm. Iran claiming to have 'warned Nigeria' to release detained Shiite leader.... @afalli

RT @cerenomri: "Literally every US ally in Mideast is on brink of hot war w/ Iran, so we're going to release $100 billion to Iran this mont…



## Exercise 2
Build a word2vec model of the tweets we have collected using gensim.
First take the collection of tweets and tokenize them using spacy.

### Exercise 2a:
* Think about how this should be done. 
* Should you only use upper-case or lower-case? 
* Should you remove punctuations or symbols? 

In [16]:
text_split = [[x.text if x.pos != spacy.parts_of_speech.VERB else x.lemma_ 
                for x in nlp_toolkit(t)] for t in tweets]


### Exercise 2b:
Build a word2vec model.
Test the window size as well - this is how many surrounding words need to be used to model a word. What do you think is appropriate for Twitter? 

In [17]:
model = Word2Vec(text_split, size=100, window=4, min_count=5, workers=4)

### Exercise 2c:
Test your word2vec model with a few similarity functions. 
* Find words similar to 'Syria'.
* Find words similar to 'war'.
* Find words similar to "Iran".
* Find words similar to 'Verizon'. 



In [22]:
model.wv.most_similar(positive=['Syria'])

KeyError: "word 'hitting' not in vocabulary"

# Exercise 2d

Adjust the choices in (b) and (c) as necessary.


## Exercise 3

Filter tweets to those that mention 'Iran' or similar entities and another word of your choosing.
* Do this using just spacy.
* Do this using word2vec similarity scores.

In [98]:
# Using spacy
for tweet in tweets:
    parsed = nlp_toolkit(tweet)
    if mentions_country(parsed, 'Iran') or mentions_country(parsed, 'Iraq'): # ... you could add more
        if 'attacking' in get_actions(parsed):
            print(tweet)

RT @f396: 'Iran has a long record in attacking foreign diplomatic missions,' Saudi ... - https://t.co/3gaSRB3osT via https://t.co/UjStGmTT2f



In [115]:
# Using word2vec similarity scores
for tweet in tweets[:200]:
    parsed = nlp_toolkit(tweet)

    similarity_to_iran = max([model.wv.similarity('Iran', tok.text) for tok in parsed if tok.text in model.wv.vocab])
    similarity_to_war = max([model.wv.similarity('war', tok.text) for tok in parsed if tok.text in model.wv.vocab])
    if similarity_to_iran > 0.99 and similarity_to_war > 0.99:
        print (tweet)
  
    

LebanonHashtag: RT probrandz: #Facebook Open Sources Artificial #Intelligence Servers Before #Google | Re/code https://t.co/f8RTJxEzSx

KalGuntuku: RT bunnymiilk "RT 30seclovelive: [RT to win!] 30 Seconds of Love Live! 5k Followers Giveaway #1: $25 iTunes/Google Play gift ca…

LebanonHashtag: RT probrandz: #Facebook Open Sources Artificial #Intelligence Servers Before #Google | Re/code https://t.co/VgWBxAG52q

RT @f396: Iran blames America, Britain and 'Zionists' for Nimr execution - https://t.co/BwXEicgAOA via https://t.co/UjStGmTT2f

RT @f396: Saudi Arabia severs diplomatic ties with Iran over embassy fire - https://t.co/r0iZugJa3v via https://t.co/UjStGmTT2f

RT @f396: 'Iran has a long record in attacking foreign diplomatic missions,' Saudi ... - https://t.co/3gaSRB3osT via https://t.co/UjStGmTT2f

